In [1]:
import sys
sys.path.append("..")  # Add the project root to Python path

import requests
from datetime import datetime, timedelta
from typing import List, Dict, Optional


from notebooks.optimize_plan import get_task_answer, update_plan, execute_task_using_new_plan, evaulate_task_answer, stackvm_host

def get_evaluation_pending_tasks(        
    start_time: Optional[datetime] = None,
    end_time: Optional[datetime] = None,
    evaluation_statuses: Optional[List[str]] = None
) -> List[Dict]:
    """
    Fetches the list of tasks pending evaluation from the API.

    Args:
        start_time (Optional[datetime]): The start time to filter tasks.
        end_time (Optional[datetime]): The end time to filter tasks.
        evaluation_statuses (Optional[List[str]]): List of evaluation statuses to filter by. Defaults to ['NOT_EVALUATED'].

    Returns:
        List[Dict]: A list of tasks pending evaluation.
    
    Raises:
        requests.exceptions.RequestException: If the request fails.
        ValueError: If the response cannot be decoded.
    """
    endpoint = f"{stackvm_host}/api/tasks/evaluation"
    params = {}
    
    if start_time:
        params['start_time'] = start_time.isoformat()
    if end_time:
        params['end_time'] = end_time.isoformat()
    if evaluation_statuses:
        # Join multiple statuses with commas
        params['evaluation_status'] = ','.join(evaluation_statuses)
    else:
        # Default to NOT_EVALUATED if no statuses are provided
        params['evaluation_status'] = 'NOT_EVALUATED'
    
    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4XX or 5XX)
        data = response.json()
        
        if not isinstance(data, list):
            raise ValueError("Unexpected response format: Expected a list of tasks.")
        
        return data
    except requests.exceptions.RequestException as e:
        # Handle network-related errors
        print(f"An error occurred while making the request: {e}")
        raise
    except ValueError as ve:
        # Handle JSON decoding errors or unexpected data formats
        print(f"An error occurred while processing the response: {ve}")
        raise

def record_evaluation(
    task_id: str,
    evaluation_status: str,
    evaluation_reason: Optional[str] = "",
    timeout: int = 60
) -> Dict:
    """
    Records the evaluation result of a specific task by calling the API endpoint.

    Args:
        base_url (str): The base URL of the API (e.g., 'http://stackvm-dev.tidb.ai:5556').
        task_id (str): The ID of the task to be evaluated.
        evaluation_status (str): The evaluation status (e.g., "APPROVED", "REJECTED").
        evaluation_reason (Optional[str]): The reason for the evaluation decision.
        api_token (Optional[str]): API token for authentication, if required.
        timeout (int): Timeout in seconds for the API request.

    Returns:
        Dict: The JSON response from the API indicating success or failure.
    
    Raises:
        requests.exceptions.RequestException: If the request fails.
        ValueError: If the response cannot be decoded or contains an error.
    """
    endpoint = f"{stackvm_host}/api/tasks/{task_id}/evaluation"
    payload = {
        "evaluation_status": evaluation_status,
        "evaluation_reason": evaluation_reason
    }
    headers = {
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(endpoint, json=payload, headers=headers, timeout=timeout)
        response.raise_for_status()
        data = response.json()

        if not isinstance(data, dict):
            raise ValueError("Unexpected response format: Expected a JSON object.")

        if not data.get("success", False):
            error_message = data.get("error", "Unknown error occurred.")
            raise ValueError(f"API Error: {error_message}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        raise
    except ValueError as ve:
        print(f"An error occurred while processing the response: {ve}")
        raise

def record_human_evaluation(
    task_id: str,
    evaluation_status: str,
    feedback: Optional[str] = "",
    timeout: int = 60
) -> Dict:
    """
    Records the evaluation result of a specific task by calling the API endpoint.

    Args:
        base_url (str): The base URL of the API (e.g., 'http://stackvm-dev.tidb.ai:5556').
        task_id (str): The ID of the task to be evaluated.
        evaluation_status (str): The evaluation status (e.g., "APPROVED", "REJECTED").
        evaluation_reason (Optional[str]): The reason for the evaluation decision.
        api_token (Optional[str]): API token for authentication, if required.
        timeout (int): Timeout in seconds for the API request.

    Returns:
        Dict: The JSON response from the API indicating success or failure.
    
    Raises:
        requests.exceptions.RequestException: If the request fails.
        ValueError: If the response cannot be decoded or contains an error.
    """
    endpoint = f"{stackvm_host}/api/tasks/{task_id}/human_evaluation"
    payload = {
        "evaluation_status": evaluation_status,
        "feedback": feedback
    }
    headers = {
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(endpoint, json=payload, headers=headers, timeout=timeout)
        response.raise_for_status()
        data = response.json()

        if not isinstance(data, dict):
            raise ValueError("Unexpected response format: Expected a JSON object.")

        if not data.get("success", False):
            error_message = data.get("error", "Unknown error occurred.")
            raise ValueError(f"API Error: {error_message}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the request: {e}")
        raise
    except ValueError as ve:
        print(f"An error occurred while processing the response: {ve}")
        raise


In [2]:
import json
from app.utils.json import extract_json

def optimize_plan(task_id:str, branch_name:Optional[str]="main", max_iteration=2):
    current_branch_name = branch_name
    error_message = None

    for i in range(2):
        print(f"Start to evaluate plan for task(id={task_id},branch={current_branch_name})")
        detail = get_task_answer(task_id, current_branch_name)

        if detail is not None:
            goal = detail.get("goal")
            final_answer = detail.get("final_answer")
            plan = detail.get("plan")
            metadata = detail.get("metadata")

            response = evaulate_task_answer(goal, metadata, final_answer, plan)
            try:
                eval_res_str = extract_json(response)
                eval_res = json.loads(eval_res_str)
            except Exception as e:
                print(f"Failed to decode evaluation result {e}: {response}")
                return

            eval_status = "APPROVED" if eval_res.get("accept", False) else "REJECTED"
            eval_reason = json.dumps(eval_res, indent=2) 
            print(f"Goal:{goal}, the evaluation result:\n{eval_res}")

            record_evaluation(task_id, eval_status, eval_reason)

            if eval_res.get("accept", False) is True:
                print(f"Goal:{goal}, Pass!\n The evaluation result:\n{eval_res}")
                return


            revised_plan_response = update_plan(goal, metadata, plan, eval_reason)

            try:
                revised_plan_str = extract_json(revised_plan_response)
                revised_plan = json.loads(revised_plan_str)
            except Exception as e:
                error_message = f"Failed to decode revised plan {e}: {revised_plan_response}"
                break

            print("revised plan:", revised_plan)

            try:
                updated_result = execute_task_using_new_plan(task_id, revised_plan)
                print(f"Revised plan execution result {updated_result}")
            except Exception as e:
                error_message = f"Failed to execute task using new plan {e}"
                break
            
            current_branch_name = updated_result.get("branch_name", None)
            current_final_answer = updated_result.get("final_answer", None)
            if current_branch_name is None or current_final_answer is None:
                error_message = "Failed to execut task using new plan, get empty answer"
                break
    
    if error_message is None:
        error_message = "Still failed after two evaluations round."
    record_human_evaluation(task_id, "WAITTING_FOR_EVALUATION", error_message)
            

    


In [3]:
from app.controller.label_classifier import LabelClassifier


optimize_plan("0eda1580-95db-4219-a23b-58336c3b8f4e", "main")

"""
classifier = LabelClassifier()

end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=2)

pending_tasks = get_evaluation_pending_tasks(
    start_time=start_time
)

for task in pending_tasks:
    task_id = task["id"]
    optimize_plan(task_id, "main")
"""


Start to evaluate plan for task(id=0eda1580-95db-4219-a23b-58336c3b8f4e,branch=main)


2025-01-13 16:37:51,714 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Goal:Optimize the following SQL query for TiDB:

```sql
select iid, title, component, html_url
from (
    select iid, title, component, html_url, pull_request_id from 
    (
        select issue_id as iid, title, component, html_url
        from dwd_component_issues 
        where type_label = 'type/bug'
            and component != 'tidb'
            and state = 'closed'
            and repo = 'tidb'
            and create_time >= '2023-10-01'
    ) as issue_table
    left join issue_pr_relation
        on issue_table.iid = issue_pr_relation.issue_id
) as fix_issue
where pull_request_id is not null
```, the evaluation result:
{'accept': False, 'answer_quality_assessment_explanation': "The final answer provides general optimization suggestions for SQL queries in TiDB, but it does not directly address the specific SQL query provided by the user. The suggestions are broad and do not include a practical, tailored optimization solution for the given query. The answer lacks a detailed analy

2025-01-13 16:38:17,109 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


revised plan: [{'seq_no': 0, 'type': 'reasoning', 'parameters': {'chain_of_thoughts': '【问题背景】\n用户希望优化一个在 TiDB 上执行的 SQL 查询。查询涉及多个嵌套子查询和连接操作，可能存在性能瓶颈。\n\n【初步假设】\n1) 查询可能缺少合适的索引，导致全表扫描。\n2) 嵌套查询和连接操作可能导致不必要的数据处理。\n3) 查询结构可以通过重写来优化。\n\n【需要检索】\n- TiDB 查询优化的最佳实践。\n- 如何分析和优化执行计划。\n- 索引设计和使用的建议。\n- 使用 EXPLAIN 语句分析给定查询的执行计划。\n- 针对 WHERE 子句和 JOIN 条件的索引建议。\n- 优化 JOIN 操作和重写查询结构的具体技术。', 'dependency_analysis': '后续步骤将分别检索 TiDB 查询优化的最佳实践、执行计划分析方法、索引设计建议，并使用 EXPLAIN 语句分析查询执行计划。最后整合所有信息以提供优化方案。'}}, {'seq_no': 1, 'type': 'calling', 'parameters': {'output_vars': ['optimization_best_practices'], 'tool_name': 'retrieve_knowledge_graph', 'tool_params': {'query': 'TiDB query optimization best practices'}}}, {'seq_no': 2, 'type': 'calling', 'parameters': {'output_vars': ['execution_plan_analysis'], 'tool_name': 'vector_search', 'tool_params': {'query': 'How to analyze and optimize SQL execution plans in TiDB', 'top_k': 5}}}, {'seq_no': 3, 'type': 'calling', 'parameters': {'output_vars': ['index_design_recommen

2025-01-13 16:39:09,803 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Goal:Optimize the following SQL query for TiDB:

```sql
select iid, title, component, html_url
from (
    select iid, title, component, html_url, pull_request_id from 
    (
        select issue_id as iid, title, component, html_url
        from dwd_component_issues 
        where type_label = 'type/bug'
            and component != 'tidb'
            and state = 'closed'
            and repo = 'tidb'
            and create_time >= '2023-10-01'
    ) as issue_table
    left join issue_pr_relation
        on issue_table.iid = issue_pr_relation.issue_id
) as fix_issue
where pull_request_id is not null
```, the evaluation result:
{'accept': True, 'answer_quality_assessment_explanation': "The final answer effectively resolves the goal by providing a comprehensive approach to optimizing the SQL query for TiDB. It includes detailed steps for execution plan analysis, identification of performance bottlenecks, and specific optimization suggestions such as adding indexes and query restructuring

'\nclassifier = LabelClassifier()\n\nend_time = datetime.utcnow()\nstart_time = end_time - timedelta(hours=2)\n\npending_tasks = get_evaluation_pending_tasks(\n    start_time=start_time\n)\n\nfor task in pending_tasks:\n    task_id = task["id"]\n    optimize_plan(task_id, "main")\n'